In [1]:
import requests, csv, io
from datetime import datetime as dt
from pprint import pprint

In [25]:
'''
# Load data externally
sheet_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRrTyONku0NgOZ2f_aFeJ8bKJ4K4wc4brHxMm9Q1_Si72bhdgefUinLCiUmoZFQg4V7F95Y4rbU0bcf/pub?gid=0&single=true&output=tsv'
response = requests.get(sheet_url)
response.encoding = response.apparent_encoding # Fix encoding (see https://stackoverflow.com/questions/44203397/python-requests-get-returns-improperly-decoded-text-instead-of-utf-8)
response.raise_for_status()
text = response.text
'''
pass

In [44]:
# Very early impersonator data file contains mostly minstrel show data

with open('./data/very early impersonators.tsv') as f:
    text = f.read()

In [34]:
headers = text.split("\n")[0]
headers = headers.split("\t")
rows = response.text.split("\n")[1:]

In [35]:
_rows = []
for row in rows:
    fields = row.split("\t")
    if fields[0] == "" or fields[0] == None: continue
    _ = {}
    for i, field in enumerate(fields):
        _[headers[i]] = field
    _rows.append(_)
rows = _rows

In [41]:
import re
for i, row in enumerate(rows):
    sources = row['Source/s'].split("\r\n")
    for source in sources:
        row['Source/s'] = row['Source/s'].replace("Archive: ", "")

        title, vol, iss, date = None, None, None, None
        g = re.search("^([A-Za-z\s.-]+) \(.*\); (.*): ", row['Source/s'])
        if g:
            title = g.groups()[0]

            g2 = re.search("(Vol\. (\d+))", row['Source/s'])
            if g2: vol = g2.groups()[0]

            g3 = re.search("(Iss\. (\d+))", row['Source/s'])
            if g3: iss = g3.groups()[0]

            g4 = re.search("((\d{2}) (Jan|Feb|Mar|Apr|May|Jun|June|Jul|July|Aug|Sep|Oct|Nov|Dec) (\d{4}))", row['Source/s'])
            if g4: 
                date_str = g4.groups()[0].replace("June", "Jun")
                date_str = date_str.replace("July", "Jul")
                date = dt.strptime(date_str, "%d %b %Y")
            else: 
                g4b = re.search("((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (\d+), (\d{4}))", row['Source/s'])
                if g4b: date = dt.strptime(g4b.groups()[0], "%b %d, %Y")

            if date is None:
                print(f"Warning: Could not find date in the following data.\n\n{row['Source/s']}\n\n\n")
        g = re.search("(.*): (.*)\.", row['Source/s'])
        if g:
            page_number = g.groups()[1]

        '''
        rows[i]['title'] = title
        rows[i]['vol'] = vol
        rows[i]['iss'] = iss
        rows[i]['date'] = date
        '''


Current Literature (1888-1912); New York Vol. VOL. XX,, Iss. No. 5, (Nov 1896): 452.




The American Law Review (1866-1906); St. Louis Vol. 38,  (Sep/Oct 1904): 751. 





In [42]:
import pandas as pd
df = pd.DataFrame.from_dict(rows[0:5])

In [43]:
df

,id,Year,First name,Last name,alt spelling of name,Company,Play/performance (mentioned),Keyword,Comment,Source/s
0,2,1854,Thomas L.,Floyd,Thomas Floyd; Thomas L. Flloyd; Thomas L. Floy...,Kunkel's Nightingale Minstrels,,,(The celebrated female impersonator of Kunkel’...,"The Sun (1837-1992); Baltimore, Md. [Baltimore..."
1,1,1866,Joseph,Maies,Joseph Mayes; Joseph Mays; Joseph Mais; Joe Ma...,Excelsior Minstrels and Burlesque Opera Troupe,,,,The Louisville Daily Journal (1839-1868); Loui...
2,3,1866,Thomas L.,Floyd,,Kunkel Opera Troupe,,,"Kunkel Opera Troupe was a ""Well-established en...",The Louisville Daily Journal (1839-1868); Loui...
3,4,1868,Joe,Maies,[? spelled Mairs in article],"Skiff & Gaylord’s Minstrels, Chicago",,,,"Detroit Free Press (1858-1922); Detroit, Mich...."
4,5,1869,R.,Saphrini,R. Safrini; R. Saffrini; R. Saffrinni; R. Safr...,,,,"as the Dashing Belle, Girl of the Period, Pet ...","The Scotsman (1860-1920); Edinburgh, Scotland ..."
